# Del 4: Sampling og den diskrete Fouriertransformasjonen

I denne delen så kommer jeg til å introdusere den diskrete Fouriertransformasjonen. Fouriertransformasjonen som vi har hittil sett på er det som kalles den kontinuerlige Fouriertransformasjonen, som er definert for kontinuerlige funksjoner. Dette skaper da problemer når vi skal implementere Fouriertransformen på en datamaskin. Datamaskiner kan kun gjennomføre regneoperasjoner på diskrete verider, og derfor må vi ha en diskret versjon av Fouriertransformasjonen som gir en diskret resultat.

Vi gjør dette i to steg. Først så tar vi for oss en diskret funksjon $x(n)$, hvor $x(n)$ er en diskret funksjon. Det er ofte tilskrekkelig å så på dette som en tallfølge (ofte representert som en array i Python). Vi må også benytte oss av summetegnet i stedet for integraltenget, siden vi nå har en diskret funksjon.

Fouriertransformasjonen vår ser nå slik ut: 

$$X(\xi) = \sum_{-\infty}^{\infty} x(n)e^{-i 2\pi \xi n}$$

Dette kalles for DTFT-en, eller diskret-tids Fouriertransformasjonen. Merk at $\xi$, altså variablen til frekvensen, er fortsatt en kontinuerlig variabel. I tillegg tar summegrensene rede for at funksjonen er uendelig, som er litt rart når vi skal jobbe med endelige tallfølger på en datamaskin. Vi diskretiserer den på følgende måte: 

$$\xi = \frac{k}{N}$$

hvor $k$ er den k-te frekvensen i $X(k)$, og vi endrer summegrensene slik at de summerer over en endelig lengde. Fouriertransformasjonen vår blir da:

$$X(k) = \sum_{n=0}^{N - 1} x(n)e^{-i\frac{2\pi nk}{N}}$$

Hvor $x(n)$ er en diskret funksjon, og $X(k)$ er den diskrete Fouriertransformasjonen av $x(n)$. $N$ er antall punkter i $x(n)$, og $k$ er den k-te frekvensen i  $X(k)$. Dette kalles for DFT-en, eller den diskrete Fouriertransformasjonen, og det er akkurat denne vi bruker når vi implementerer Fouriertransformasjonen på en datamaskin.

Prinsippet til DFT-en er da akkurat den samme som med den kontinuerlige Fouriertransformasjonen: vi tar en eller annen funksjon som befinner seg i tidsdomenet, og transformerer den til frekvensdomenet. Allikevel så er det noen nyanser med DFT-en som vi må være klar over. For at vi skal kunne forstå oss på disse nyansene, så må vi først se på hvordan man diskretiserer en kontinuerlig funksjon, også kalt sampling:

## Sampling



Sampling er når vi tar en kontinuerlig funksjon og gjør den diskret. Intuitivt så kan vi si at vi deler opp den kontinuerlige funksjonen i biter med lik intervall mellom seg, og så tar vi verdien til funksjonen i disse punktene: 

<img src="plots/p4_cont.png" width="400px"> <img src="plots/p4_cont_samp.png" width="400px"> <img src="plots/p4_samp.png" width="400px"> 

Etter vi har gjort dette har vi en diskret tallrekke som kan lagres på og bli behandlet av en datamaskin. 

La oss introdusere noen nye begreper: 

- $N$: Total antall samplepunkter i den diskrete funksjonen. 
- Samplingsintervall: Også kjent som samplingsperiode. Dette er avstanden mellom hvert samplepunkt, og er ofte betegnet med $T_s$, med sekunder som enhet.
- Samplingsfrekvens: Dette er antall samplepunkter per sekund, og er ofte betegnet med $f_s$, med enheten Hz. Samplingsfrekvensen er da den inverse av samplingsintervallet: $f_s = \frac{1}{T_s}$

Hvis vi da tar for oss en sinusfunksjon som vi pleier å se i Python:
```python
t = np.linspace(0, 10, 1000) # tid fra 0 til 10 sekunder med 1000 samplepunkter
sinus = np.sin(t)
```

er $N = 1000$, samplingsintervallet er $T_s = 10/1000 = 0.01$ sekunder, og samplingsfrekvensen er $f_s = 1/T_s = 100$ Hz.

Konverteringen fra kontinuerlig til diskret er en prosess som kalles for anolog til digital konvertering, og er noe som skjer på hardwarenivå, for eksempel når du har en mikrofon som tar inn en lydbølge (en anolog / kontinuerlig verdi) og konverterer den til en digital tallrekke ved å sample den. Vi gjør noe lignende i kodesnutten overnfor, hvor vi definerer en diskret tallrekke med N antall samplingspunkter med linspace-funksjonen, og bruker det som input til sinusfunksjonen. Da får vi en diskret sinusfunksjon.


## Undersampling og Nyquists samplingsteorem

Hva skjer hvis vi har en kontinuerlig funksjon, og vi prøver å sample den med en samplingsfrekvens som er for lav? La oss ta sinusfunksjonen fra kodesnutten ovenfor som en eksempel:

```python
t = np.linspace(0, 10, 1000) 
sinus = np.sin(t)
```

Hvis vi plotter den rett ut så ser den slik ut:

<img src="plots/p4_sine.png" width="400px">

Ser ut som en helt vanlig sinusfunksjon. La oss nå minke samplingsfrekvensen, altså øke samplingsintervallet til noe banalt: 

```python
t = np.linspace(0, 10, 3) 
sinus = np.sin(t)
```

Vi har altså en sinusfunksjon med en frekvens på $2\pi$ Hz, som vi skal prøve å sample med kun tre punkter fordelt på 10 sekunder. Samplingsintervallet er nå $T_s = 10/3$ sekunder, og samplingsfrekvensen er $f_s = 1/T_s = 3/10$ Hz. Hvis vi plotter denne funksjonen så får vi:

<img src="plots/p4_sine_usamp.png" width="400px">

Dette ser jo ikke ut som en sinusfunksjon i det hele tatt! Her kan vi bemerke at plottet er særdeles stygg for å være en sinusfunksjon, og at vi mister informasjon om hvor ofte funksjonen svinger. I det "fine" plottet ser vi tydelig at funksjonen svinger ca. 1.5 ganger i løpet av 10 sekunder, mens ut ifra det stygge plottet så kan vi fort tro at den svinger mindre enn en gang i løpet av samme tid.

Dette kalles for undersampling. Vi har altså en funksjon med en eller annen frekvenskomponent, i dette tilfellet kun en frekvenskomponent med frekvens lik $2\pi$ Hz, og vi sampler den med en samplingsfrekvens som er for lav. Dette fører til at vi mister informasjon om frekvenskomponentene til funksjonen, og vi får en helt annen funksjon enn den vi hadde i utgangspunktet. 

Men hva er "for lav" samplingsfrekvens? Altså, hva er den laveste samplingsfrekvensen vi kan ha før vi begynner å miste informasjon om funksjonen? Da er vi inne på **Nyquists samplingsteorem**, som sier at for å unngå undersampling, så **må samplingsfrekvensen være minst dobbelt så stor som den høyeste frekvenskomponenten i funksjonen**. Den høyeste frekvensen som man kan sample uten å miste informasjon kalles for Nyquist-frekvensen, som er da halvparten av samplingsfrekvensen.

Hvis vi tar for oss en funksjon som er en sum av to sinusfunksjoner med frekvenser 1 Hz og 2 Hz, så må samplingsfrekvensen være minst 4 Hz for å unngå undersampling. Med en samplingsfrekvens på 4 Hz så vil vi få en Nyquist-frekvens på 2 Hz. Om vi sampler med en samplingsfrekvens på 10 Hz så vil Nyquist-frekvensen være 5 Hz, altså den høyeste frekvenskomponenten vi kan sample uten å miste informasjon. Hvis vi derimot sampler med en samplingsfrekvens på 3 Hz, vil Nyquist-frekvensen være 1.5 Hz, altså lavere enn den høyeste frekvenskomponenten i funksjonen, og vi vil miste informasjon om funksjonen.

## Konsekvenser av diskretisering i frekvensdomenet 

Så langt så har vi sett på sampling og undersampling i tidsdomenet. Men har undersampling, og det at vi i det hele tatt har en diskret funksjon og Fouriertransformasjon, konsekvenser i frekvensdomenet? Svaret er ja, og det er akkurat dette som skiller resultatet til DFT-en fra den kontinuerlige Fouriertransformasjonen. 

### Periodisering

En av konsekvensene av å diskretisere Fouriertransformasjonen og funksjonen vi transformerer er at DFT-en "ser" funksjonen den skal transformere som en periodisk funksjon, uansett om den faktisk er det eller ikke, og at en diskret funksjon kommer alltid til å ha en periodisk frekvensdomene: 

<img src="plots/cont_disc_spectra.gif" width="800px">

*Forholdet mellom den kontinuerlige Fouriertransformasjonen og den diskrete Fouriertransformasjonen. Fra* *[Wikipedia](https://en.wikipedia.org/wiki/Discrete_Fourier_transform)* *, av bruker Sbyrnes321.*

*Venstre kolonne: En kontinuerlig funksjon (øverst) og dens Fourier-transformasjon (nederst).* 

*Midt-venstre kolonne: En periodisk funksjon (øverst). Fourier-transformasjonen (nederst) er null unntatt ved diskrete punkter, altså den består av en endelig sum av frekvenskomponenter.*

*Midt-høyre kolonne: Den opprinnelige funksjonen (øverst) er diskretisert (samplet). Vi tar dens Fouriertransformasjon med DTFT-en, og dens Fourier-transformasjon (nederst) er en periodisk summering av den opprinnelige transformasjonen. Med andre ord: frekvensdomenet til en diskret funksjon er alltid periodisk*

*Høyre kolonne: Vi tar DTFT-en til den opprinnelige funksjonen og diskretiserer den, og dette blir da en DFT (nederst). På grunn av en rekke med matematiske konsekvenser som jeg ikke kommer til å gå i dybden i, blir den inverse DFT-en (øverst) en periodisk summering av den opprinnelige aperiodiske funksjonen. Dette er hva DFT-en "ser" når den transformerer en funksjon til frekvensdomenet. Hver periode DFT-en "ser" kommer til å være like lang som antall samplepunkter i den diskrete funksjonen.*

Oppsumert: selv om en kontinuerlig funksjon er aperiodisk og har en aperiodisk frekvensdomene, så kommer dens frekvensdomene til å bli periodisk når vi tar Fouriertransformasjonen av en diskret versjon av den. I tillegg så "ser" DFT-en alle funksjoner som en periodisk funksjon, uansett om vi i utgangspunktet tar en aperiodisk funksjon. Kort fortalt så er dette fordi i motsetning til DTFT-en, som summerer fra og til uendeligheten og dermed tar rede for en uenderlig diskret funksjon, så summerer DFT-en kun over en endelig lengde, og dermed må den anta at funksjonen er periodisk.

Avstanden mellom hver periode i frekvensdomenet er samplingsfrekvensen:

**INSERT IMAGE HERE**

Så hva skjer vi vi nå undersampler, og avstanden mellom hver periode i frekvensdomenet blir mindre enn Nyquist-frekvensen? Da vil vi få noe som kalles for aliasing.

### Aliasing

Aliasing skjer når vi undersampler en signal, og periodene i frekvensdomenet begynner å overlappe hverandre. Følgelig så får vi villedende informasjon om hvilke frekvenskomponenter inngår i funksjonen.